In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import mnist
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
class Network:

    def __init__(self) -> None:
        pass

    # load the dataset (MNIST)
    def load_dataset(self):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = mnist.load_data()
        # Convert grayscale images to RGB by repeating the single channel three times
        self.x_train = np.stack((self.x_train,)*3, axis=-1)
        self.x_test = np.stack((self.x_test,)*3, axis=-1)

        # Resize images to 48x48 to fit VGG16 input requirements
        self.x_train = tf.image.resize(self.x_train, (48, 48)).numpy()
        self.x_test = tf.image.resize(self.x_test, (48, 48)).numpy()

        self.x_train = self.x_train.astype('float32') / 255.0
        self.x_test = self.x_test.astype('float32') / 255.0

        self.y_train = to_categorical(self.y_train, 10)
        self.y_test = to_categorical(self.y_test, 10)

        # Split training data for cross-validation
        self.x_train2, self.x_val, self.y_train2, self.y_val = train_test_split(self.x_train, self.y_train, test_size=0.2, random_state=42)



    # show one instance of each class
    def show_instances(self):
        class_instances = []
        for i in range(10):
            class_instance = None
            for j, label in enumerate(self.y_train.argmax(axis=1)):
                if label == i:
                    class_instance = self.x_train[j]
                    break
            class_instances.append(class_instance)
        
        plt.figure(figsize=(12, 6))
        for i, img in enumerate(class_instances):
            plt.subplot(2, 5, i+1)
            plt.imshow(img.astype('float32').squeeze(), cmap='gray')
            plt.title(f'Class: {i}')
            plt.axis('off')
        plt.show()


    # create model with given parameters
    def create_model(self, dropout_rate=0.5):
        model = Sequential()
        # Load the VGG16 model, excluding the top fully connected layers
        try:
            # Load the VGG16 model, excluding the top fully connected layers
            vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
        except Exception as e:
            print(f"Exception: {e}")
            # Download VGG16 weights manually
            vgg16_base = VGG16(weights=None, include_top=False, input_shape=(48, 48, 3))
            vgg16_base.load_weights(tf.keras.utils.get_file(
                'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                cache_subdir='models'))

        # Freeze the convolutional base
        for layer in vgg16_base.layers:
            layer.trainable = False

        # Build the model
        model.add(vgg16_base)
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(dropout_rate))
        model.add(Dense(10, activation='softmax'))

        # Compile the model
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        self.model = model
        self.model.summary()
        

    # train the best model on the training set and report train, validation and test results
    def train_eval(self, dropout=0.5):
        self.create_model(dropout_rate=dropout)
        history = self.model.fit(self.x_train, self.y_train, validation_split=0.2, epochs=1, batch_size=200, verbose=1)

        # Evaluate the model on the training, validation, and test data
        train_loss, train_accuracy = self.model.evaluate(self.x_train, self.y_train, verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.x_val, self.y_val, verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.x_test, self.y_test, verbose=0)

        print(f'Train accuracy: {train_accuracy}')
        print(f'Validation accuracy: {val_accuracy}')
        print(f'Test accuracy: {test_accuracy}')

        # Plotting the training history
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Loss over epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Accuracy over epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.show()

        # Displaying the results in a table
        results = {
            'Dataset': ['Train', 'Validation', 'Test'],
            'Loss': [train_loss, val_loss, test_loss],
            'Accuracy': [train_accuracy, val_accuracy, test_accuracy]
        }

        results_df = pd.DataFrame(results)
        print(results_df)

    def eval(self):
        # Evaluate the model on the training, validation, and test data
        train_loss, train_accuracy = self.model.evaluate(self.x_train, self.y_train, verbose=0)
        val_loss, val_accuracy = self.model.evaluate(self.x_val, self.y_val, verbose=0)
        test_loss, test_accuracy = self.model.evaluate(self.x_test, self.y_test, verbose=0)

        print(f'Train accuracy: {train_accuracy}')
        print(f'Validation accuracy: {val_accuracy}')
        print(f'Test accuracy: {test_accuracy}')

        # Plotting the training history
        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Train Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Loss over epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'], label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Accuracy over epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.show()

        # Displaying the results in a table
        results = {
            'Dataset': ['Train', 'Validation', 'Test'],
            'Loss': [train_loss, val_loss, test_loss],
            'Accuracy': [train_accuracy, val_accuracy, test_accuracy]
        }

        results_df = pd.DataFrame(results)
        print(results_df)



In [3]:
net = Network()
net.load_dataset()
net.show_instances()

ResourceExhaustedError: {{function_node __wrapped__ResizeBilinear_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[60000,48,48,3] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:ResizeBilinear] name: 

In [ ]:
net.train_eval()



Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5: 403 -- Forbidden
58889256/58889256 [==============================] - 180s 3us/step

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                 

MemoryError: Unable to allocate 264. MiB for an array with shape (10000, 48, 48, 3) and data type float32